In [11]:
cur.close()

In [1]:
import polars as pl
import numpy as np
import psycopg2

In [12]:
import polars as pl
import numpy as np
import psycopg2

connection = psycopg2.connect(user="postgres",
                              # пароль, который указали при установке PostgreSQL
                              password="",
                              host="localhost",
                              port="5432",
                              database="HH_contest_db")
cur = connection.cursor()


In [15]:
import numpy as np

for row in test.iter_rows():
    print(1)
    user_vec = np.array(list(range(300)))
    cur.execute(f"SELECT vacancy_id from user_data where user_id = '{row[0]}' and action_type = '1'")
    viewed_vacs = [i[0] for i in
                   cur.fetchall()]  #получили все просмотренные пользователем вакансии #[('u_482520', 's_17494976', 'v_39399', '2', '2023-11-01T16:36:10.950000'), (), ...]
    user_towns_set = set()  #тут будут все города
    viewed_vacs_set = set()  #тут все вакансии с которыми
    print(2)
    if viewed_vacs:  #проверка на пустоту
        print(row[0])
        cur.execute(
            f"select * from sum_values_of_vacs(ARRAY (SELECT vacancy_id from user_data where user_id = '{row[0]}' and action_type = '1'))")
        user_vec = np.array(cur.fetchall()[0][0])
        user_vec = list(user_vec / np.sqrt(np.sum(user_vec ** 2)))
        cur.execute(
            f"select predicted_group, area_id, name from joined_vacs_predicted_groups where vacancy_id in (SELECT vacancy_id from user_data where user_id = '{row[0]}' and action_type = '1')")
        unpacked_data = list(zip(*cur.fetchall()))
        viewed_groups = str(list(set(unpacked_data[0]))).replace("[", "(").replace("]", ")")
        viewed_towns = str(list(set(unpacked_data[1]))).replace("[", "(").replace("]", ")")
        viewed_names = str(list(set(unpacked_data[2]))).replace("[", "(").replace("]", ")")
        viewed_vacs_set = str(list(set(viewed_vacs))).replace("[", "(").replace("]", ")")
        print('viewed_towns:', viewed_towns)
        print("Отклики:", viewed_names, viewed_vacs)

        cur.execute(f"""
        select 
            vacancy_id, 
            predicted_group, 
            name, 
            area_id, 
            cosine_similarity(sum_values_of_vacs( array (SELECT vacancy_id from user_data where user_id = '{row[0]}' and action_type = '1')), get_values_by_id(vacancy_id)) 
        from joined_vacs_predicted_groups 
        where area_id in {viewed_towns} and
            vacancy_id not in {viewed_vacs_set} and 
            predicted_group in {viewed_groups} and 
            cosine_similarity(sum_values_of_vacs( array (SELECT vacancy_id from user_data where user_id = '{row[0]}' and action_type = '1')), get_values_by_id(vacancy_id)) > 0.90
        order by cosine_similarity(sum_values_of_vacs( array (SELECT vacancy_id from user_data where user_id = '{row[0]}' and action_type = '1')), get_values_by_id(vacancy_id)) desc""")

        data = zip(*cur.fetchall())
        print(pl.DataFrame(data)[['column_0', 'column_1', 'column_2', 'column_3', 'column_4', 'column_5']])
        #   print("vac_info:", vac_info)
    #    print("user_towns_set", user_towns_set)
    #    print('viewed_vacs', viewed_vacs_set)
    #    break
    else:
        print('такого нет в базе')
        continue


1
2
u_482520
viewed_towns: ('a_3095', 'a_3445')
Отклики: ('Супервайзер по производственной безопасности (HSE)', 'Ведущий специалист по охране труда', 'Специалист по охране труда / Начальник службы охраны труда', 'Главный специалист по ОТ и ПБ (Качканар)', 'Специалист по охране труда и промышленной безопасности', 'Главный специалист отдела охраны труда', 'Специалист по охране труда службы производственной безопасности и производственного контроля') ['v_593279', 'v_916144', 'v_625926', 'v_953483', 'v_2310302', 'v_2702260', 'v_1844906']
shape: (5, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ column_0       ┆ column_1       ┆ column_2       ┆ column_3      ┆ column_4      ┆ column_5      │
│ ---            ┆ ---            ┆ ---            ┆ ---           ┆ ---           ┆ ---           │
│ str            ┆ str            ┆ str            ┆ str           ┆ str           ┆ str           │
╞════════════════╪════════════════╪═══════

KeyboardInterrupt: 

In [21]:
test = pl.read_parquet(r"hh_recsys_test_hh.pq")

In [35]:
cur.execute("select * from get_values_by_id('v_983456')")

In [7]:
data = pl.read_csv(r"normalized_vacs.csv")

In [11]:
cols = list(data.columns)
cols.remove('vacancy_id')
data[cols].shape

(2734129, 131)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

# Замените X на ваш набор данных многомерных векторов
X = data[cols]

# Укажите количество кластеров (замените на желаемое значение)
n_clusters = 1000

# Создайте и обучите модель MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=1000)
kmeans.fit(X)

# Получите метки кластеров для каждого вектора
cluster_labels = kmeans.labels_

# Выведите результат
print("Метки кластеров для первых 10 векторов:", cluster_labels[:10])

In [7]:
data = pl.read_csv(r"normalized_vacs_predicted_groups.csv")

In [10]:
data = data[['vacancy_id', 'predicted_group']]

In [14]:
data

vacancy_id,predicted_group
str,i64
"""v_2556094""",127
"""v_2514145""",58
"""v_615464""",108
"""v_401606""",42
"""v_979935""",786
"""v_94226""",927
"""v_2060705""",958
"""v_2517136""",453
"""v_1977652""",108


In [13]:
data.write_csv('normalized_vacs_predicted_groups.csv')

In [21]:
data = data.with_columns(pl.Series(name='predicted_group', values=cluster_labels))

In [22]:
data.sort(by='predicted_group')

vacancy_id,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,…,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,predicted_group
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""v_2086680""",0.040016,-0.052243,0.005551,0.019074,-0.034188,0.013751,0.025161,0.033766,0.088542,0.095712,-0.048417,-0.060194,0.027788,0.048906,-0.056451,-0.00262,-0.100694,-0.107983,-0.057483,0.02948,0.081396,-0.00744,0.021441,-0.006688,-0.054658,-0.026474,-0.143774,0.122529,-0.040548,-0.042385,-0.079234,-0.078896,0.126558,0.019064,-0.012813,-0.065528,…,-0.017246,-0.031916,-0.069079,-0.059232,-0.214799,0.025051,0.008279,-0.106067,-0.011526,0.006908,0.093485,-0.075214,0.000417,-0.071274,0.039184,0.00486,-0.019307,-0.077936,0.020549,-0.030218,0.030392,-0.02128,0.027312,-0.016709,0.066739,0.186853,0.061593,-0.034507,0.005607,0.0924,-0.021349,-0.069273,0.020746,-0.031783,0.016098,-0.001554,0
"""v_959665""",0.025834,-0.018251,0.005207,-0.000709,-0.031387,-0.003843,-0.018649,0.032915,0.12511,0.139759,-0.063585,-0.088601,0.039744,0.041765,-0.045825,0.012053,-0.073894,-0.043497,-0.054422,0.040066,0.063087,-0.013822,-0.025499,0.004457,-0.034753,-0.029427,-0.125427,0.146294,-0.025512,-0.05056,-0.066138,-0.093556,0.118457,0.017852,-0.009351,-0.060447,…,-0.009633,-0.056614,-0.030995,-0.01309,-0.182939,-0.005126,-0.002387,-0.095199,0.001456,-0.011144,0.062744,-0.085464,0.010623,-0.056933,0.066817,0.005512,0.002429,-0.101887,0.023374,-0.017998,0.050302,-0.039442,0.034641,-0.050581,0.063378,0.155749,0.016816,-0.017723,0.021688,0.071101,-0.037702,-0.102349,0.064272,-0.040948,0.014574,-0.024246,0
"""v_1344348""",0.048708,-0.043198,0.066693,0.023282,-0.022744,0.003269,0.038381,0.016775,0.10974,0.077729,-0.05233,-0.05846,-0.001234,0.041376,-0.055523,0.012451,-0.098329,-0.125426,-0.055388,-0.000411,0.065274,-0.031106,-0.037364,-0.02231,-0.068815,-0.020657,-0.127037,0.131066,-0.043803,-0.046798,-0.079484,-0.067928,0.108878,0.023552,-0.000125,-0.050374,…,-0.016732,-0.048629,-0.090072,-0.057981,-0.200291,0.006921,0.050485,-0.074102,-0.010209,0.00071,0.093731,-0.073173,-0.004286,-0.057479,0.048613,-0.000166,-0.035644,-0.062215,0.012271,-0.065571,0.019784,-0.046815,0.04542,-0.034384,0.047549,0.166495,0.026731,-0.032506,-0.036849,0.06909,-0.032628,-0.091966,0.014333,-0.038992,0.012307,0.007437,0
"""v_2689157""",0.074744,-0.063884,-0.009344,0.02162,0.003051,-0.014034,0.009489,-0.003859,0.113289,0.068447,-0.050312,-0.072902,0.044075,0.017104,-0.061719,0.031744,-0.088183,-0.112546,-0.069561,0.017557,0.019271,-0.059581,-0.015642,-0.030489,-0.098809,0.009334,-0.138198,0.087966,-0.051753,-0.056246,-0.061287,-0.061144,0.178724,0.058849,-0.013223,-0.062588,…,0.015337,-0.043712,-0.046967,-0.058263,-0.175328,0.005642,-0.013876,-0.096078,-0.01741,-0.003281,0.070301,-0.073457,-0.004753,-0.064719,0.017474,0.009897,-0.037388,-0.100968,0.041514,-0.042715,0.042866,-0.039085,0.005192,0.008518,0.051786,0.162919,0.046658,-0.009942,-0.00299,0.075548,-0.102193,-0.08158,-0.003057,-0.00787,0.009523,0.017589,0
"""v_59341""",0.028683,-0.041293,-0.000819,0.019208,-0.019686,-0.023711,0.00104,0.011573,0.126909,0.095416,-0.036927,-0.091355,-0.00086,0.008746,-0.062996,0.030681,-0.07721,-0.095346,-0.073128,-0.01004,0.046776,-0.030172,-0.035432,-0.019226,-0.076305,-0.037617,-0.126202,0.110238,-0.023563,-0.029934,-0.096168,-0.082421,0.123068,0.018788,0.004669,-0.033953,…,-0.003924,-0.031009,-0.04574,-0.065533,-0.1784

In [4]:
cur.execute("select name from joined_vacs where vacancy_id in ('v_2613538', 'v_1144785', 'v_1022634', 'v_471731')")

In [5]:
cur.fetchall()

[('Оператор фасовочной линии',),
 ('Оператор линии сборки',),
 ('Оператор линии на производстве',),
 ('Оператор линии',)]

In [26]:
import psycopg2
from concurrent.futures import ThreadPoolExecutor

# Connection parameters
db_params = {
    'dbname': 'your_database_name',
    'user': 'postgres',
    'password': '',
    'host': 'localhost',
    'port': '5432',
}


def execute_query(user_id):
    try:
        with psycopg2.connect(user="postgres",
                              # пароль, который указали при установке PostgreSQL
                              password="",
                              host="localhost",
                              port="5432",
                              database="HH_contest_db") as connection:
            with connection.cursor() as cursor:
                # Execute the SQL query for the given user ID
                cursor.execute(f"""
                   with user_data_table as (
            select joined_vacs_predicted_groups.area_id as a_i, joined_vacs_predicted_groups.vacancy_id as v_i, joined_vacs_predicted_groups.predicted_group as p_i
            from joined_vacs_predicted_groups
            join otklick_table on joined_vacs_predicted_groups.vacancy_id = otklick_table.vacancy_id
            WHERE otklick_table.user_id = '{user_id}')
        select
            vacancy_id,
            predicted_group,
            name,
            area_id,
            cosine_similarity(sum_values_of_vacs( array (SELECT v_i from user_data_table)), get_values_by_id(vacancy_id)) as similarity
        from joined_vacs_predicted_groups
        where vacancy_id not in (select v_i from user_data_table) and predicted_group in (select p_i from user_data_table) and area_id in (select a_i from user_data_table)
        order by similarity desc;
                """)
                result = cursor.fetchall()
                return result
    except Exception as e:
        return f"Error executing query for user {user_id}: {str(e)}"


# List of user IDs
user_ids = needed_users[:10]

# Create a ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    # Execute queries asynchronously for each user ID
    results = list(executor.map(execute_query, user_ids))

# Process the results
for user_id, result in zip(user_ids, results):
    print(f"Results for user {user_id}:", result)

Results for user u_482520: [('v_2719071', 128, 'Ведущий специалист по охране труда и промышленной безопасности', 'a_3445', 0.9817030134175891), ('v_836178', 128, 'Специалист по охране труда', 'a_3445', 0.9796559619280505), ('v_2006025', 128, 'Специалист по охране труда', 'a_3445', 0.9791475766174428), ('v_1058860', 269, 'Главный специалист по охране труда и промышленной безопасности', 'a_3095', 0.9791014737185257), ('v_1019015', 269, 'Специалист по охране труда и промышленной безопасности', 'a_3095', 0.9789679495345774), ('v_1015691', 269, 'Главный специалист по охране труда и промышленной безопасности', 'a_3445', 0.9789128369392216), ('v_801136', 269, 'Главный специалист по охране труда и промышленной безопасности', 'a_3445', 0.9789128369392216), ('v_172564', 128, 'Старший специалист по охране труда', 'a_3095', 0.9783940893071293), ('v_2214225', 128, 'Специалист по охране труда', 'a_3095', 0.9781442536665548), ('v_1230121', 269, 'Специалист по охране труда и промышленной безопасности'

In [ ]:
dataframe = pl.DataFrame()

In [61]:
answers = []
for i in range(1, 21):
    file = open(rf'C:\Users\byari\PycharmProjects\pythonProject4\posgres_python_files\output_files\{i}.txt', encoding='utf-8').read().split(';;;')[:-1]
    try:
        for answer in file:
            user = answer.split(' : ')[0].replace('\n', '')
            exec('user_data = '+ ''.join(answer.split(' : ')[1:]).replace('\n', ''))
            answers.append([user, list(i[0] for i in user_data)[:5]])
    except Exception:
        print(answer)
    

In [62]:
dataframe = pl.DataFrame({'user_id':list(zip(*answers))[0], 'predictions':list(zip(*answers))[1]})

In [63]:
submit_sample = pl.read_parquet(r"hh_recsys_sample.pq")[['user_id', 'session_id']]

In [64]:
submit_sample = submit_sample.join(dataframe, on='user_id')

In [59]:
submit_sample

user_id,session_id,predictions
str,str,list[str]
"""u_482520""","""s_25018731""","[""v_219359"", ""v_394683"", … ""v_1223061""]"
"""u_582132""","""s_481216""","[""v_46272"", ""v_699397"", … ""v_683693""]"
"""u_212584""","""s_16918781""","[""v_764241"", ""v_698860"", … ""v_2134327""]"
"""u_425177""","""s_17505104""","[""v_822732"", ""v_1554381"", … ""v_884927""]"
"""u_700997""","""s_15528830""","[""v_2645145"", ""v_385031"", … ""v_2406160""]"
"""u_687201""","""s_11187326""","[""v_2727306"", ""v_36966"", … ""v_586782""]"
"""u_573219""","""s_27394215""","[""v_795895"", ""v_1674789"", … ""v_586541""]"
"""u_1082782""","""s_30341164""","[""v_689933"", ""v_1260082"", … ""v_2297568""]"
"""u_81444""","""s_1909964""","[""v_1934927"", ""v_2518872"", … ""v_1380185""]"


In [65]:
submit_sample.write_parquet('first_answer.pq')